# Dependencies

In [2]:
import pandas as pd
import numpy as np
from constraint import *
import random

In [3]:
DISTANCE_DATASET_PATH = "distances.xlsx"
INDEX_VALUES_DATASET_PATH = "index_values.csv"
INITIAL_DISTRIBUTION_DATASET_PATH = "initial_distribution.txt"
N_SELLERS = 4
N_BRICKS = 22

# Dataset loader

In [17]:
class Dataset:
    def __init__(self, distance_dataset_path, index_values_dataset_path, initial_distribution_dataset_path):
        # Store dataset path
        self.distance_dataset_path = distance_dataset_path
        self.index_values_dataset_path = index_values_dataset_path
        self.initial_distribution_dataset_path = initial_distribution_dataset_path
        
        # Read dataset
        self.read_distances()
        self.read_index_values()
        self.read_initial_distribution()
    
    def read_distances(self):
        """
        Load and store the data from the excel sheet provided in the object
        """
        # Load first sheet 
        brick_bureau = pd.read_excel(self.distance_dataset_path, skiprows=[0], sheet_name=0, header=0, index_col=1, dtype=str)
        # Drop empty cells of the first column
        brick_bureau = brick_bureau.drop(labels=brick_bureau.columns[0], axis=1)
        
        # Load second sheet
        brick_brick = pd.read_excel(self.distance_dataset_path, sheet_name=1, header=1, index_col=1, dtype=str)
        # Drop emty cells of the first column
        brick_brick = brick_brick.drop(labels=brick_brick.columns[0], axis=1)
        
        # Store in self
        self.brick_bureau = brick_bureau
        self.brick_brick = brick_brick
        
        
    def read_index_values(self):
        """ Read index values, les_v
        """
        self.index_values = np.genfromtxt(self.index_values_dataset_path, delimiter=',')
    
    def read_initial_distribution(self):
        """ Read initial state of bricks distribution betweek sellers: les_a
        """
        with open(self.initial_distribution_dataset_path, "r") as file:
            raw_data = file.read().split("\n")
            # remove header
            raw_data.pop(0)
            initial_distribution = []
            for line in raw_data:
                line.replace(" ", "")
#                 id_seller = line.split(":")[0]
                les_id_bricks = line.split(":")[1].split(",")
                initial_distribution.append(les_id_bricks)
        
        les_a = np.zeros((N_SELLERS, N_BRICKS))
        for id_seller in range(len(initial_distribution)):
            for id_brick in initial_distribution[id_seller]:
                les_a[id_seller, int(id_brick)-1] = 1
        
        self.initial_affectation = les_a
        
    
    def disp_dataset(self):
        for dataset in [self.brick_brick, self.brick_brick, self.index_values, self.initial_affectation]:
            print("")
            print("="*20)
            print(dataset)
    
        
        
dataset = Dataset(DISTANCE_DATASET_PATH, INDEX_VALUES_DATASET_PATH, INITIAL_DISTRIBUTION_DATASET_PATH)

In [18]:
dataset.disp_dataset()


                    1                   2                   3   \
1                    0   7.353257781419065  13.208754672564709   
2    7.353257781419065                   0    6.57540873254279   
3   13.208754672564709    6.57540873254279                   0   
4    16.15966893225229  19.000328944520934   25.29384312436526   
5   18.219385280519205  20.060199400803572   26.05085027403136   
6    16.68325208105422  19.064367285593303   25.24229981598349   
7   14.003831618524982   16.30070243885214  22.536115459413143   
8    19.02224487277987  21.588353341558964   27.75903816777519   
9    17.44701693700101   21.44988578058168  27.928322541821235   
10   28.48672146808053  30.986004905440783   36.98608522133696   
11  27.201332687940123   29.89193369456048   35.96907699677599   
12  29.304260782350404  25.599150376526172  27.896754291494197   
13  24.528583326396983  25.984799402727745  31.680356689911182   
14  24.094136216100377  26.473492402779048   32.49800301557005   
15  25.38

# Problem definition

VARIABLES:
- $x_{ij}$ ; = 1 if seller `i` is allocated to brick `j`, 0 otherwise.  

PARAMETERS:
- $d_{ij}$ ; distance from office of seller `i` to the brick `j`  
- $v_j$ ; index value of the `j` brick
- $a_{ij}$ ; = 1 if seller `i` was originally allocated to brick `j`, 0 otherwise.  


## Minimize total distance traveled

We want to minimize the total traveled distance, then we want to minimize:  
$\underset{i=1}{\overset{4}{\sum}}\underset{j=1}{\overset{22}{\sum}}x_{ij}d_{ij}$  

## Minimize imbalance in workload

We want to minimize the maximum of difference between the workers. They have to share 1/4 of the sum of the index values of bricks. Then, we want to minimize the absolute difference between this value and the seller workload.  
This way, we want to minimize:  
$\underset{i\in\{1, 2, 3, 4\}}{max}\left| \frac{1}{4} \underset{j=1}{\overset{22}{\sum}} v_j - \underset{j=1}{\overset{22}{\sum}} x_{ij}v_j\right|$

## Minimize distruption

We want to minimize the changes from the previous model to the new one, leading to minimizing:  
$\underset{i=1}{\overset{4}{\sum}}\underset{j=1}{\overset{22}{\sum}}x_{ij}(1-a_{ij})v_j$

- x: Matrix of shape number_of_seller=4 * number_of_bricks=22 with 0 or 1
- a: Same shape than x
- d: Matrix of shape number_of_seller=4 * number_of_bricks=22 with float

In [37]:
class Cost:
    
    def __init__(self, les_d, les_v, les_a):
        self.les_d = les_d
        self.les_v = les_v
        self.les_a = les_a
        self.total_workload = self.get_total_workload()
        
    
    def get_total_workload(self):
        """ Return total workload computed using les_v, the index value of each brick
        """
        return np.sum(self.les_v)
        
        
    def cost_distance(self, les_x):
        """ Return cost regarding distances
        :param les_x: matrix numpy, with x[i, j] set to 1 if seller i have to cover brick j
        
        :return cost: cost value regarding distances
        """
        return np.sum(les_x * self.les_d)
    
    
    def cost_imbalance_worload(self, les_x):
        """ Return cost regarding imbalance workload
        :param les_x: matrix numpy, with x[i, j] set to 1 if seller i have to cover brick j
        
        :return cost: cost value regarding imbalance workloard
        """
        
        les_workload = [np.abs(0.25*self.total_workload - np.sum(les_x_seller * self.les_v)) for les_x_seller in les_x]
        return np.max(les_workload)
    
    def cost_disruption(self, les_x):
        """ Return cost regarding disruption"""
        return np.sum(les_x * ( 1 - self.les_a )*self.les_v )
    
        
    

In [38]:
class Interface:
    
    def __init__(self):
        self.dataset = Dataset(DISTANCE_DATASET_PATH, INDEX_VALUES_DATASET_PATH, INITIAL_DISTRIBUTION_DATASET_PATH)
        self.les_d = self.get_les_d()
        self.les_v = self.get_les_v()
        self.les_a = self.get_les_a()
        self.les_x_test = self.generate_les_x_test()
        
        self.coster = Cost(les_d=self.les_d, les_v=self.les_v, les_a=self.les_a)

    
    def get_les_d(self):
        """ Return value of distances betweek sellers office and bricks
        """
        les_d = np.array(self.dataset.brick_bureau, dtype=np.float).transpose()
        return les_d
    
    
    def get_les_v(self):
        """ Return index values of brickes, with les_v[j] = index value of brick j
        """
        les_v = self.dataset.index_values.reshape((1, -1))
        return les_v

    
    def get_les_a(self):
        """ Return les a"""
        les_a = self.dataset.initial_affectation
        return les_a
    
    
    def generate_les_x_test(self):
        """
        To test cost function, we need to generate a false affectation of seller to bricks.
        Thus, we create a random affectation.
        """
        # Create a x_test matrix, inversed shape, with line corresponding to bricks, columns to sellers
        les_x_test = np.zeros((N_BRICKS, N_SELLERS))
        
        # Put one 1 per line, to affect each brick to a seller
        for id_line in range(len(les_x_test)):
            id_one = random.randint(0, len(les_x_test[id_line]) - 1)
            les_x_test[id_line, id_one] = 1
        
        # Lets transpose the matrix, as we want seller as rows.
        les_x_test = les_x_test.transpose()
        return les_x_test
    
    
    def test_cost_distance(self):
        print(self.coster.cost_distance(les_x=self.les_x_test))
        
        
    def test_cost_imbalance_worload(self):
        print(self.coster.cost_imbalance_worload(les_x=self.les_x_test))
        
    def test_cost_distruption(self):
        print(self.coster.cost_disruption(les_x=self.les_x_test))
    
interfacer = Interface()
interfacer.test_cost_distance()
interfacer.test_cost_imbalance_worload()
interfacer.test_cost_distruption()

476.15999999999997
0.8979
3.401
